In [1]:
import tfim
import tfim_perturbation
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy import optimize
import progressbar
import itertools

In [2]:
marker = itertools.cycle((',', '+', '.', '1','2','3','4','|')) 

In [3]:
# Initial system specification
L = [4]
Jij_seed = 19
h_x_range = np.arange(0, 0.01, 0.0002)

In [4]:
PBC = True
J = 1

In [5]:
# Build lattice and basis
###################################
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)
###################################

#construct random J matrix
Jij = tfim.Jij_instance(N,J,"bimodal",Jij_seed) 

In [6]:
# List out all the spin_states, corresponding indices and energies
Energies = -tfim.JZZ_SK_ME(basis,Jij)
for index in range(2**N):
    print(index, basis.state(index), Energies[index])

0 [0 0 0 0] -0.0
1 [0 0 0 1] -2.0
2 [0 0 1 0] -2.0
3 [0 0 1 1] -0.0
4 [0 1 0 0] 6.0
5 [0 1 0 1] -0.0
6 [0 1 1 0] -0.0
7 [0 1 1 1] -2.0
8 [1 0 0 0] -2.0
9 [1 0 0 1] -0.0
10 [1 0 1 0] -0.0
11 [1 0 1 1] 6.0
12 [1 1 0 0] -0.0
13 [1 1 0 1] -2.0
14 [1 1 1 0] -2.0
15 [1 1 1 1] -0.0


In [14]:
# Build 2nd order approximated matrix

GS_energy, GS_indices = tfim_perturbation.GS(Energies)

print(GS_energy)

H_app_0 = tfim_perturbation.H_app_0(GS_energy, GS_indices)

H_app_1 = tfim_perturbation.H_app_1(basis, GS_indices, N)

H_app_2 = tfim_perturbation.H_app_2(basis, Jij, GS_indices, N, GS_energy)

-2.0


In [12]:
# Build exact matrix
V_exc = tfim_perturbation.V_exact(basis, lattice)

H_0_exc = tfim_perturbation.H_0_exact(Energies)

In [13]:
# Calculate approximated eigenvalues and eigenstates for range(h_x)
app_eigenvalues, app_eigenstates = tfim_perturbation.app_2_eigensystem(GS_indices, GS_energy, h_x_range, J, N, basis, Jij)